# Credmark Modeling Framework Example for Jupyter notebook
## Introduction

version: 2023.4.15

## Installation

If you would like to use Conda environment, run below commands in the terminal.

    NAME=cmk # rename it to what you want to
    conda create -y --name "$NAME"
    conda install -y --name "$NAME" pip
    conda activate ${NAME}
    conda install -y --name "$NAME" ipython jupyterlab nb_conda_kernels black isort
    conda install -y --name "$NAME" jupyterlab_code_formatter ipywidgets
    conda install -y --name "$NAME" scipy numpy
    conda install -y --name "$NAME" altair pandas matplotlib openpyxl xlsxwriter
    conda activate ${NAME}

Launch Jupyter notebook and change the kernel in the newly created environment and run following commands.

They will download the git repository of Cremark models in the directory two-levels up `../../` which is on the same level of `credmark-models-notebook` (line 1) and install the Credmark Modeling Framework (line 2).

    !cd ../../ && git clone https://github.com/credmark/credmark-models-py.git
    !cd ../../credmark-models-py && pip install -r requirements.txt && pip install -e .


## Initialize

### 1. Create context and some shortcuts for frequently used utilitis.

* Load the ipython extension

* Create the initialization parameters
    - chain_id: 1 for ETH
    - block_number: None or a specific number.
    - chain_to_provider_url: Web3 node (archive is preferred)
    - api_url: None or a specific gateway server address
    - use_local_models: None, '-' '*', or a comma-separated list.
    - register_utility_global: True (set global variables for various utilities, e.g. ledger, web3, etc.)

* Call the extension to create the context

<div class="alert alert-block alert-info">
    <b>Note:</b> Change to a web3 provider you have in `params`/`chain_to_provider_url` below.
</div>

In [1]:
%reload_ext credmark.cmf.ipython

param = {'chain_id': 1,
 'block_number': None,
 'chain_to_provider_url': {'1': 'https://eth-mainnet.g.alchemy.com/v2/4HREehwJhXq-fkm1llCDGol-swXEXj0L'},
 'api_url': None,
 'use_local_models': None,
 'register_utility_global': True}

context, model_loader = %cmf param

* Get help for Cmf extension

In [ ]:
%cmf help

In [ ]:
%cmf help_param

In [ ]:
%cmf default_param

### 2. (Optional) adjust logging level

You could change the level of logging with `log_output()`.

In [ ]:
import logging
# Change output of logging to a file, and 
# Change the level to DEBUG when no level is specified.
# logging_output('../tmp/debug.log')

# Change level of logging from the default (WARNING) to
# either INFO (less information)
# or DEBUG(more information).
# Still output to stream.
log_output(log_level=logging.INFO)

## Use Cmf

### 1. Basic utilities

In [ ]:
get_dt(2022, 5, 3)

In [ ]:
get_block(get_dt(2022, 5, 3))

In [ ]:
context.block_number

In [ ]:
context.chain_id

In [ ]:
context.web3

In [ ]:
context.web3.eth.get_balance('0xd3CdA913deB6f67967B99D67aCDFa1712C293601')

In [ ]:
context.web3.eth.get_block(context.block_number)['gasUsed']

### 2. Use models

1. Type `models.` and hit `TAB` key, a list of models will be pop up.
2. Select a model and type ? will show description and help, `models.token.info?`
3. Show the list of models wiht `dir(models)`, or `model_loader.loaded_model_versions()` with version information.
4. If you have updated any model in the folder on disk, run `model_loader.reload()` to refresh.

In [ ]:
dir(models)

In [ ]:
models.tokens.erc20?

In [ ]:
model_loader.reload()

## 3. Create and use types

### 3.1 Example - get price for USDC

In [ ]:
usdt = Token(address='0xdAC17F958D2ee523a2206206994597C13D831ec7')
print('USDT decimals:', usdt.decimals)
models.chainlink.price_by_ens({'domain': 'usdt-usd.data.eth'})

In [ ]:
models(get_block(get_dt(2022, 5, 3))).chainlink.price_by_ens({'domain': 'usdt-usd.data.eth'})

In [ ]:
models(get_block(get_dt(2022, 5, 3))).chainlink.price_by_registry(base=usdt)

### 3.1 Run a model

#### Run as of current block

In [ ]:
aave = Token(symbol='AAVE')

print(block_number,
      block_number.timestamp_datetime,
      'models:', context.models.price.quote(base=aave, return_type=Price).price,
      'run_model:', context.run_model('price.quote', input={'base': aave}, return_type=Price).price)

#### Run as of past blocks

In [ ]:
context.block_number.timestamp_datetime

In [ ]:
for dt in [get_dt(2022, 5, 3), get_dt(2022, 5, 9), get_dt(2022, 1, 1)]:
    block = get_block(dt)
    print(block,
          block.timestamp_datetime,
          'models:', context.models(block).price.quote(base=aave, return_type=Price).price,
          'run_model:', context.run_model('price.quote', input={'base': aave}, return_type=Price, block_number=block).price)

### 3.2 Travel between different blocks

Tavel is one-way only - can only travel to earlier/smaller block numbers, and not to the future/later blocks.

In [ ]:
context.block_number = get_block(get_dt(2022, 5, 3))
models.price.quote(base=aave, return_type=Price).price,

In [ ]:
# Below will gave any error as we wants to travel back
# context.block_number = get_block(get_dt(2022, 5, 30))
# models.chainlink.price_usd(input=aave, return_type=Price).price

Instead, create a new context. It will automatically become the current context.

In [ ]:
params_old = param | {'block_number': get_block(get_dt(2022, 4, 3))}

context_old,_ = %cmf params_old
context_old.block_number.timestamp_datetime, context.block_number.timestamp_datetime

If we would like to use previously defined context, run `context.set_current()` to set it back to the current context.

Otherwise, we may encounter error with running model for a block number later than the context.

In [ ]:
context.set_current()

### 4. Ledger

In [ ]:
with context.ledger.Transaction as q:
    df_ledger = (q.select(
        columns=[q.BLOCK_HASH,
                 q.FROM_ADDRESS,
                 q.TO_ADDRESS,
                 q.VALUE],
        where=q.BLOCK_NUMBER.eq(context.block_number-1000),
        order_by=q.BLOCK_TIMESTAMP,
        limit=5)
             .to_dataframe())
df_ledger

In [ ]:
with context.ledger.Transaction as q:
    df_ledger = (q.select(
        columns=[q.BLOCK_HASH,
                 q.FROM_ADDRESS,
                 q.TO_ADDRESS,
                 q.VALUE],
        where=q.BLOCK_NUMBER.eq(context.block_number-1000),
        order_by=q.BLOCK_TIMESTAMP,
        limit=5)
        .to_dataframe())
df_ledger

Bad pipe message: %s [b'~\x90\x82H9\x8c\x04\x0e*\xc5$\xc9E3Z\x825\x95 \xae\xd0\x1bQ\x04\x8e\xada\n\x15_^"\xe3\xb3\xd7\x13\x8b,oP\x94\xca\x03\xd2\xa66\xdc{Sh\x81\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08']
Bad pipe message: %s [b'\x05\x08\x06']
Bad pipe message: %s [b'\x05\x01\x06', b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \x8bm&Gx\xec\xc6<\xe7\xc9\xb0i\x0e\x92\xb2\x9e\xb4\xca\xc75\x9cy']
Bad pipe message: %s [b"5vL\x02\xc0#\x8a\xf1(\xb4\xfd2l\x8c\xbe\x1f&\xd8\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00